Prediction

In [88]:
import tensorflow as tf
import pickle
import pandas as pd
from tensorflow.keras.models import load_model
import numpy as np

In [89]:
# Load the trained model
model = load_model("Model.h5")

# Load the pickle file
with open("label_encoder_gender.pkl","rb") as file:
    label_encoder_gen = pickle.load(file)
with open("geo_encoder_location.pkl","rb") as file:
    geo_encoder_location = pickle.load(file)
with open("scalar_dataset.pkl","rb") as file:
    scaler = pickle.load(file)

In [90]:
type(geo_encoder_location)

sklearn.preprocessing._encoders.OneHotEncoder

In [91]:
# Example Input Data
input_data = {
    'CreditScore': 569,
    'Geography': 'France',
    'Gender':'Male',
    'Age':42,
    'Tenure': 3,
    'Balance':63000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':5000
}

In [92]:
input_data

{'CreditScore': 569,
 'Geography': 'France',
 'Gender': 'Male',
 'Age': 42,
 'Tenure': 3,
 'Balance': 63000,
 'NumOfProducts': 2,
 'HasCrCard': 1,
 'IsActiveMember': 1,
 'EstimatedSalary': 5000}

In [93]:
geo_encoded = geo_encoder_location.transform([[input_data['Geography']]]).toarray()
# geo_encoded = geo_encoder_location.transform([input_data['Geography']])
geo_location = pd.DataFrame(geo_encoded,columns=geo_encoder_location.get_feature_names_out(['Geography']))
geo_location

c:\Users\SANDE\Documents\New Projects - ML\condavenv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [94]:
input_data = pd.DataFrame([input_data])
input_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,569,France,Male,42,3,63000,2,1,1,5000


In [95]:
input_data['Gender']= label_encoder_gen.transform(input_data['Gender'])
input_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,569,France,1,42,3,63000,2,1,1,5000


In [96]:
# concat
input_data = pd.concat([input_data.drop("Geography",axis=1),geo_location],axis=1)
input_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,569,1,42,3,63000,2,1,1,5000,1.0,0.0,0.0


In [97]:
# Scaler the input
input_scaled = scaler.transform(input_data)
input_scaled

array([[-0.85769482,  0.91324755,  0.29493847, -0.69539349, -0.20977822,
         0.80843615,  0.64920267,  0.97481699, -1.65923237,  1.00150113,
        -0.57946723, -0.57638802]])

In [99]:
prediction = model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 31ms/step


array([[0.04365161]], dtype=float32)

In [106]:
if prediction[0][0] > 0.50:
    print(f'Model has predicted churn with accuracy: {prediction[0][0]}')
else:
    print(f"Model has predicted no churn with accuracy: {1 - prediction[0][0]:.4f}")


Model has predicted no churn with accuracy: 0.9563
